## Hosted MCP Tool with Microsoft Agent Framework

![mslearn_mcp_server](./images/mslearn_mcp_server.png)

In [ ]:
%pip install agent-framework==1.0.0b251001 python-dotenv azure-ai-projects==1.1.0b4

### Setting Up the Environment

In [ ]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

### Creating our MS Learn MCP Tool Object

In [ ]:
from agent_framework import HostedMCPTool

ms_learn_mcp_tool = HostedMCPTool(
    name = "Microsoft Learn MCP Tool",
    url = "https://learn.microsoft.com/api/mcp"
)

### Creating our MCP Agent

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

# Cell 2: Define async workflow
async def create_docs_agent():
    # Create async Azure credential
    credential = AzureCliCredential()
    
    # Initialize the Azure AI Agent Client
    chat_client = AzureAIAgentClient(async_credential=credential,
                                     project_endpoint=project_endpoint,
                                     model_deployment_name=model)

    try:

        # Create the Docs Agent
        agent = chat_client.create_agent(
            name="DocsAgent",
            instructions="You are a helpful assistant that can help with Microsoft documentation questions.",
            tools=ms_learn_mcp_tool
        )

        print("✅ Agent created successfully!")
        return agent

    finally:
        # Clean up async clients
        await chat_client.close()
        await credential.close()


In [ ]:
# Cell 3: Run the async function
agent = await create_docs_agent()


### Creating an Async Function to Handle User Approvals

In [ ]:
from agent_framework import ChatMessage

async def handle_approvals_with_thread(query: str, agent, thread):
    result = await agent.run(query, thread=thread, store=True)
    while len(result.user_input_requests) > 0:
        new_input = []
        for user_input_needed in result.user_input_requests:
            print(
                f"\n⚙️ Function call requested: {user_input_needed.function_call.name}"
                f"\nArguments: {user_input_needed.function_call.arguments}"
            )
            user_approval = input("Approve function call? (y/n): ")
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(user_approval.lower() == "y")],
                )
            )
        result = await agent.run(new_input, thread=thread, store=True)
    return result


In [ ]:
thread = agent.get_new_thread()
query = "How to create an Azure storage account using az cli?"
response = await handle_approvals_with_thread(query, agent, thread)

print("\n🧠 Assistant:\n", response)
